<a href="https://colab.research.google.com/github/mahdiehhojabri/predicting-Boston-house-prices/blob/main/Predicting_Boston_House_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Loading Data

from keras.datasets import boston_housing
import numpy as np

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [ ]:
train_data.shape

(404, 13)

In [ ]:
test_data.shape

(102, 13)

In [ ]:
#Z-score Normalization, First step : Turninging the mean of each column into zero(mean=0)
mean = train_data.mean(axis=0)

train_data -=mean

 # Z-score Normalization, Second step: Turning std of each column into 1(std=1)
std = train_data.std(axis=0)
train_data /=std

test_data -=mean
test_data /=std

In [ ]:
#Network architecture
from keras import models
from keras import layers
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],))) #train_data.shape[1]=number of columns, train_data.shape[0]=number of rows
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [ ]:
#K-fold Cross Validation. We use this method for training, because there is not very big datafor validation & Training
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

#prepares the validation data :data from partition #k
for i in range(k):
    print('processing fold #%d' % i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

#Prepares the training data : data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

#Builds the keras model(already compiled)
    model = build_model()

#Trains the model(in silent mode,verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    
#Evaluates the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold #0
processing fold #1
processing fold #2
processing fold #3


In [ ]:
print(all_scores)

[2.0192668437957764, 2.545945644378662, 2.493720293045044, 2.298738479614258]


In [ ]:
np.mean(all_scores)

2.339417815208435